In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [14]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch! hello python"
chars =  "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

In [15]:
char_list = [i for i in chars]
n_letters = len(char_list)

In [16]:
print(n_letters)

35


In [23]:
# 원핫 인코더
def string_to_onehot(string):
    start = np.zeros(shape = n_letters, dtype = int)
    end = np.zeros(shape = n_letters, dtype = int)

    start[-2] = 1
    end[-1] = 1
        #이렇게 해주는 이유가, 전에꺼를 다음꺼로 넘겨주기 위함.
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape = n_letters, dtype = int)
        
        zero[idx] = 1
        start = np.vstack([start, zero])

    output = np.vstack([start, end])
    return output

In [24]:
#위에 원핫 만들어준거 다시 문자로 바꾸는 함수 생성.
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [25]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i20 = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1) #입력, 은닉층을 합치고,
        hidden = self.act_fn(self.i2h(combined)) #이걸 활성화 함수 통과,
                                                 
        output = self.i20(combined) # 결과값 계산하게끔.
        return output, hidden

        #은닉층 초기화.
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [26]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

In [27]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    optimizer.zero_grad()
    hidden = rnn.init_hidden()

    total_loss = 0

    for j in range(one_hot.size()[0] -1): #맨 마지막 글자는 넣을 필요 x 그래서 -1
        input_ = one_hot[j:j+1, :]
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_, hidden)

        loss = loss_fn(output.view(-1), target.view(-1))
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(1.2001, grad_fn=<AddBackward0>)
tensor(0.2819, grad_fn=<AddBackward0>)
tensor(0.1078, grad_fn=<AddBackward0>)
tensor(0.0627, grad_fn=<AddBackward0>)
tensor(0.0441, grad_fn=<AddBackward0>)
tensor(0.0351, grad_fn=<AddBackward0>)
tensor(0.0311, grad_fn=<AddBackward0>)
tensor(0.0294, grad_fn=<AddBackward0>)
tensor(0.0287, grad_fn=<AddBackward0>)
tensor(0.0285, grad_fn=<AddBackward0>)
tensor(0.0284, grad_fn=<AddBackward0>)
tensor(0.0281, grad_fn=<AddBackward0>)
tensor(0.0356, grad_fn=<AddBackward0>)
tensor(0.0307, grad_fn=<AddBackward0>)
tensor(0.0293, grad_fn=<AddBackward0>)
tensor(0.0287, grad_fn=<AddBackward0>)
tensor(0.0282, grad_fn=<AddBackward0>)
tensor(0.0327, grad_fn=<AddBackward0>)
tensor(0.0295, grad_fn=<AddBackward0>)
tensor(0.0286, grad_fn=<AddBackward0>)
tensor(0.0283, grad_fn=<AddBackward0>)
tensor(0.0273, grad_fn=<AddBackward0>)
tensor(0.0341, grad_fn=<AddBackward0>)
tensor(0.0288, grad_fn=<AddBackward0>)
tensor(0.0359, grad_fn=<AddBackward0>)
tensor(0.0290, grad_fn=<A

In [28]:
start = torch.zeros(1, n_letters)
start[:, -2] = 1 # 마지막 문자 다음에 오는 문자를 시작 문자로 지정하려고.

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start #첫 입력으로 start 전달.
    output_string = "" #나오는 문자 계속 붙여주기.

    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data) #위에꺼 누적.
        input_ = output #이번 결과값, 다음 입력값으로.

print(output_string)

hello pytorch! hello python
